## 数据集的准备

Kaggle比赛，测试集数据也会填充

底层各有优缺点，

仲裁者，模型融合

好的基石，数据，数据决定上限

上面都是 ~ 手法

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('datas/house_data.csv')
train = data

In [4]:
# train.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,SqrtLotArea
0,0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,...,None,None,None,0,2,2008,WD,Normal,208500,91.923882
1,1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,...,None,None,None,0,5,2007,WD,Normal,181500,97.979590
2,2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,...,None,None,None,0,9,2008,WD,Normal,223500,106.066017
3,3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,...,None,None,None,0,2,2006,WD,Abnorml,140000,97.724101
4,4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,...,None,None,None,0,12,2008,WD,Normal,250000,119.415242


In [ ]:
y = train['SalePrice']

In [ ]:
train1 = train.drop(['Id', 'SalePrice'], axis=1)

In [ ]:
X = pd.get_dummies(train1).reset_index(drop=True)

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
# 内容全部数字化了
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2,random_state=123)

## 基础线性回归



生成对抗网络，生成、对抗网络

deep state

输入噪声，生成图片

对抗模型：图片分类

来自`银行和印假钞`模型——画家

最早的对抗模型

评判，

知道银行柜员，不能识别，

来来回回对抗模型

生成假钞，做假钞的能力也很强

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
lm=LinearRegression()

In [ ]:
p=lm.fit(X_train, y_train)

In [ ]:
lm.score(X_test, y_test)

In [ ]:
pred=lm.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(np.log(y_test), np.log(pred)))

In [ ]:
np.sqrt(mean_squared_error(y_test, pred))

In [ ]:
def benchmark(model):
    pred = model.predict(X_test)
    logrmse = np.sqrt(mean_squared_error(np.log(y_test), np.log(pred)))
    return logrmse

In [ ]:
benchmark(lm)

## 数据预处理 Preprocessing

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
lm_model = make_pipeline(RobustScaler(), LinearRegression())

In [ ]:
lm_model.fit(X_train, y_train)

In [ ]:
# 评测模型
benchmark(lm_model)

## RidegeRegression

朴素的Ridge回归

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge_model = Ridge(alpha=0.1)

In [ ]:
ridge_model.fit(X_train, y_train)

In [ ]:
benchmark(ridge_model)

## 朴素的Ridge回归

In [ ]:
ridge_model_pipe=make_pipeline(RobustScaler(), Ridge(alpha=0.1))

In [ ]:
ridge_model_pipe.fit(X_train, y_train)

In [ ]:
benchmark(ridge_model_pipe)

## 带有CV的回归

cross view data

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kfolds=KFold(n_splits=10, shuffle=True, random_state=123)

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
r_alphas=[0.01, 0.1, 1, 3, 5, 7, 10, 100]

In [ ]:
ridge_model_cv=make_pipeline(RobustScaler(), RidgeCV(alphas=r_alphas, cv=kfolds))

In [ ]:
# RidgeCV()

In [ ]:
ridge_model_cv.fit(X_train, y_train)

In [ ]:
benchmark(ridge_model_cv)

In [ ]:
def benchmark2(model, X_test, y_test):
    pred=model.predict(X_test)
    if pred[pred<0.].shape[0]>0:
        print('Neg Value')
    rmse=np.sqrt(mean_squared_error(y_test, pred))
    lrmse = np.sqrt(mean_squared_error(np.log(y_test), np.log(pred)))
    
    print('RMSE:', rmse)
    print('LRMSE:', lrmse)

In [ ]:
benchmark2(ridge_model_cv, X_test, y_test)

In [ ]:
r_alphas=[.0001, .0003, .0005, .0007, .0009, .01, 0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 20, 30, 50, 60, 70, 80]

In [ ]:
def ridge_train_test(alpha):
    m = make_pipeline(RobustScaler(), RidgeCV(alphas=[alpha], cv=kfolds))
    m.fit(X_train, y_train)
    benchmark2(m, X_test, y_test)
    lrmse=np.sqrt(mean_squared_error(np.log(y_test),np.log(pred)))
    return lrmse

In [ ]:
ridge_train_test(.0001)

In [ ]:
# 要写很多很多野代码
scores=[]
for k in r_alphas:
    scores.append(ridge_train_test(k))

In [ ]:
plt.plot(r_alphas, scores)